# FY 2015 WIC Potential Eligibles Calculation

# Data Sources

This report calculates WIC Potential Eligibles for FY 2015. It was created by the Office of Program Decision Support (raa), FCHS, DSHS 1/20/2016.

Eligibles are calculated using a Modified CPS (Current Population Survey) Based Option. The CPS Based Option is described in:
<br/>
Estimating Eligibility and Participation for the WIC Program:  Final Report (2003).  Eds. M. Ver Ploeg & D.M. Betson.  The National Academies Press, Washington, D.C., accessed from www.nap.edu on December 31, 2016.
<br/><br/>
This 2015 calculation follows the prodedure outlined in "CPS Based Option_updated 2011 estimates.doc", created by Jamie Clark on 7/21/2014 and last modified by Dorothy Mandell on 7/21/2014.


Data for children under five is from the Texas State Demographer (http://osd.texas.gov/Data/TPEPP/Projections/). Data is projected for 2015 with full 2000–2010 migration rate. Populations of children of 0, 1, 2, 3, and 4 years were returned for each Texas county.
<br/><br/>
Texas and county projections, Population Estimates and Projections Program, Texas State Data Center, Office of the State Demographer, Institute for Demographic and Socioeconomic Research, The University of Texas at San Antonio. Data retrieved by Rachel Asquith on 12/31/2015.


Estimates of population below 185% of Federal Poverty Level are from the U.S. Census Bureau.
<br/><br/>
U.S. Census Bureau; American Community Survey, 2014 American Community Survey 5-Year Estimates, Table B17024; generated by Rachel Asquith; using American FactFinder; <http://factfinder.census.gov>; (1/7/2016)
<br/><br/>
U.S. Census Bureau; American Community Survey, 2014 American Community Survey 5-Year Estimates, Table C17002; generated by Rachel Asquith; using American FactFinder; <http://factfinder.census.gov>; (1/7/2016)

# Tools

All calculations were completed using Python and performed and presented through a Jupyter, Ipython notebook:
<br/><br/>
Python Software Foundation. Python Language Reference, version 2.7. Available at http://www.python.org
<br/><br/>
Fernando Pérez, Brian E. Granger, IPython: A System for Interactive Scientific Computing, Computing in Science and Engineering, vol. 9, no. 3, pp. 21-29, May/June 2007, doi:10.1109/MCSE.2007.53. URL: http://ipython.org
<br/><br/><br/>
<u>Dependencies:</u>
<br/><br/>
<i>numpy 1.9.2</i>
<br/>
Stéfan van der Walt, S. Chris Colbert and Gaël Varoquaux. The NumPy Array: A Structure for Efficient Numerical Computation, Computing in Science & Engineering, 13, 22-30 (2011), DOI:10.1109/MCSE.2011.37 (publisher link)
<br/><br/>
<i>pandas 0.16.2</i>
<br/>
Wes McKinney. Data Structures for Statistical Computing in Python, Proceedings of the 9th Python in Science Conference, 51-56 (2010) (publisher link)

# Weights

<script>
    MathJax.Hub.Config({
                displayAlign: 'left', // Change this to 'left' to left-align equations.
        });
</script>

The subsequent outline is copied almost verbatim from the "CPS Based Option_updated 2011 estimates.doc"; 2011 adjustment factors were removed and small changes were made for clarity. The applied adjustment factors are declared in the code blocks and were updated from: http://www.fns.usda.gov/sites/default/files/ops/WICEligibles2013-Volume2.pdf. An additonal weight from this update was added to the 2011 methodology to adjust for nutritional risk for pregnatn women (Pregnant Women D).
<br/><br/>
The following five sub populations are factored into an overall estimation of the WIC eligible population:
<br/>
•	Infants  <br/>
•	Children  <br/>
•	Pregnant women  <br/>
•	Postpartum women  <br/>
•	Breastfeeding women  <br/>

The formulas for the five populations are:
<br/><br/>
\begin{equation}Infants: A*B*C*D*E*F\end{equation}
o	A:  The number of infants less than one year of age  <br/> 
o	B:  Adjustment factor to account for the undercount of infants (12 months/11 months)  <br/>
o	C:  Proportion &lt;185 Federal Poverty Level  <br/>
o	D:  Adjustment factor to account for infants adjunctively eligible (Medicaid, TANF, and food stamps)  <br/>
o	E:  Adjustment factor to adjust for variation in monthly income <br/>
o	F:  Adjustment factor to adjust for nutritional risk
<br/><br/>
\begin{equation}Children:  A*B*C*D*E*F\end{equation}
o	A:  Number of children aged 1-4 years old  <br/>
o	B:  Adjustment factor to account for the undercount of infants (12 months/11 months)  <br/>
o	C:  Proportion &lt;185 Federal Poverty Level  <br/>
o	D:  Adjustment factor to account for children adjunctively eligible (Medicaid, TANF, and food stamps)  <br/>
o	E:  Adjustment factor to adjust for variation in monthly income <br/>
o	F:  Adjustment factor to adjust for nutritional risk
<br/><br/>
\begin{equation}Pregnant Women:  A*B*C*D\end{equation}
o	A:  Number of eligible infants calculated above  <br/>
o	B:  Adjustment factor to adjust for variation in monthly income during pregnancy and first year postpartum  <br/>
o	C:  Adjustment factor to account for fetal deaths, infant deaths and multiple births  <br/>
o	D:  Adjustment factor to adjust for nutritional risk
<br/><br/>
\begin{equation}Postpartum Women:  A*B*C\end{equation}
o	A:  Number of eligible infants calculated above  <br/>
o	B:  Adjustment factor to account for fetal deaths, infant deaths and multiple births <br/>
o	C:  Adjustment factor to estimate the number of women less than 6 months postpartum who do not breastfeed
<br/><br/>
\begin{equation}Breastfeeding Women:  A*B*C\end{equation}
o	A:  Number of eligible infants calculated above <br/>
o	B:  Adjustment factor to account for fetal deaths, infant deaths and multiple births <br/>
o	C:  Adjustment factor to estimate the number of women less than 12 months postpartum who do breastfeed


# Calculation

These first few cells import the necessary libraries, specify the filepath, and set up some formatting.

In [53]:
import pandas as pd
import numpy as np
pd.set_option("display.precision", 4)  # Specifies display precision

<strong>Step 1:</strong> First specify the file paths

In [54]:
def define_file_paths(acs_years):
    '''Create the filepaths for the input data
    Args:
        acs_years(int): 1 or 5
    Returns:
        filepaths(str)'''

    # Points to the CSV with data for children under 5
    county_under5 = fpath + 'TxSDC2014PopPrjctn_Full_Migration'
    state_under5 = county_under5 + '_State.csv'
    county_under5 += '.csv'

    # Points to data on income by ratio of FPL by age
    county_under185 = fpath + 'ACS_14_%sYR_B17024' % str(acs_years)
    state_under185 = county_under185 + '_State.csv'
    county_under185 += '.csv'

    return(county_under5, state_under5, county_under185, state_under185)

<strong>Step 2:</strong> Read, clean, and format the data for projections of children under 5.

<i>The following code performs these actions:
<br/>
1) Loads the data, selects the desired columns, cleans the input  <br/>
2) Reshapes the data  <br/>
3) Adds the total population under 5 and the total population betewen 1 and 4  <br/>
4) Converts the "Area Code" from the Texas State Demographer projection data into a FIPS code</i>

In [55]:
def preprocess(df, FIPS_col, sum_col_name, sum_col_filter, drop_col_filter, cols_to_drop=[]):
    '''The following function processes the ACS ratio of FPL data frames. It renames and cleans the FIPS column.
    It sums all the estimate columns with FPL ratio groups. The it drops all the margin of error columsn, all the ratio
    group columns besides the sums, and any other specified drop columns.
    Args:
        df (pandas dataframe): dataframe with ACS ratio data with groups of interest
        FIPS_col (string): name of the FIPS column
        sum_col_name (string): desired name of the sum column, e.g. under 185% FPL
        sum_col_filter (string): sum columns with this string in the name
        drop_col_filter (string): drop columns with this string in the name
        cols_to_drop (list of strings): additional columns to drop
    Returns:
        cleaned up dataframe'''

    df = df.rename(columns={FIPS_col: 'FIPS'})
    df['FIPS'] = df['FIPS'].apply(lambda x: str(x).strip())

    cols_to_sum = [
        col for col in df.columns if drop_col_filter not in col and sum_col_filter in col]
    df[sum_col_name] = df[cols_to_sum].sum(axis=1, numeric_only=True)

    return df.drop(cols_to_drop + [col for col in df.columns if drop_col_filter in col] + cols_to_sum, axis=1, inplace=False)

<strong>Step 3:</strong> Create the dataframe

<i>The following code performs these actions:
<br/>
1) Creates the data frames  <br/>
2) Sets up a filter for migration  <br/>
3) Merges, filters, and reshapes the data  </i>

In [56]:
def create_data_frame(fpath, migration, acs_years):
    # Create the data frames
    county_under5, state_under5, county_under185, state_under185 = define_file_paths(
        acs_years)

    # Set up a filter value for migration
    if migration == 'full':
        migration_filter = '2000-2010'
    elif migration == 'half':
        migration_filter = '0.5 2000-2010'
    else:
        migration_filter = '0 : No Migration'

    # Load the county level data
    county_under5_df = pd.read_csv(county_under5, header=0,
                                   thousands=None,
                                   dtype={'Age': np.str_})
    # And the state level data; clean the State Code
    state_under5_df = pd.read_csv(state_under5, header=0,
                                  thousands=None,
                                  dtype={'Age': np.str_}).dropna()
    state_under5_df['Area Code'] = 0

    # Merge or replace the county-level data
    if acs_years < 5:
        under5_df = state_under5_df
    else:
        under5_df = pd.concat(
            [county_under5_df, state_under5_df], axis=0, join='outer', ignore_index=True)

    # Filter by Migration
    under5_df = under5_df[under5_df['Migration Rate'] == migration_filter]
    # Format the Population
    under5_df['Total Pop'] = under5_df['Total Pop'].replace(
        regex=True, to_replace=r'\D', value=r'').astype(float)

    # Reshape the data
    under5_df = pd.pivot_table(
        under5_df, values='Total Pop', index=['Area Code', 'Area Name'], columns=['Age'])
    under5_df.columns.name = None
    under5_df = under5_df.reset_index()

    # Add the total population under 5 and the total population betewen 1 and 4
    one_to_four = ['1', '2', '3', '4']
    under5_df['total <5'] = under5_df.sum(
        axis=1, numeric_only=True)  # sums all numeric columns
    # sums only 1, 2, 3, 4
    under5_df[
        '1-4 years'] = under5_df[one_to_four].sum(axis=1, numeric_only=True)

    # Rename the columns
    replacements = dict(zip(one_to_four + ['0', 'Area Code'],
                            [str(col) + ' year' for col in under5_df[one_to_four].columns] + ['infants', 'FIPS']))
    under5_df = under5_df.rename(columns=replacements)

    # Converts the "Area Code" from the Texas State Demographer projection
    # data into a FIPS code
    under5_df['FIPS'] = under5_df['FIPS'].apply(
        lambda x: '48' + str(x).zfill(3))

    # Load and reshape the ACS data
    under185_df = pd.read_csv(county_under185, skiprows=1).iloc[:, :21]
    state_under185_df = pd.read_csv(state_under185, skiprows=1).iloc[:, :21]
    state_under185_df['Id2'] = 48000
    if acs_years < 5:
        under185_df = state_under185_df
    else:
        under185_df = pd.concat(
            [under185_df, state_under185_df], axis=0, join='outer', ignore_index=True)

    # Clean the data
    under185_df = preprocess(under185_df, FIPS_col='Id2', sum_col_name='Under 185% FPL <6',
                             sum_col_filter='Under 6 years: -', drop_col_filter='Error',
                             cols_to_drop=['Id'])

    # Merge both sets
    combined_df = pd.merge(under185_df, under5_df, how='inner', on='FIPS')
    combined_df['Prop Below 185% FPL <6'] = combined_df[
        'Under 185% FPL <6']*1./combined_df['Estimate; Under 6 years:']
    combined_df['Prop Below 185% FPL <6'].fillna(0, inplace=True)
    combined_df = combined_df[['FIPS', 'Geography', 'Estimate; Total:', 'Prop Below 185% FPL <6',
                               'infants', '1-4 years']]
    combined_df.columns = ['FIPS', 'County', 'Total Pop', 'Prop Below 185% FPL <6',
                           'infants', '1-4 years']


    # Return the data set
    return combined_df


<strong>Step 4:</strong> Apply the adjustment factors and calculate the final eligibles. Round the results to the nearest whole person.

<i>The following code performs these actions: <br/>
1) Selects the desired columns  <br/>
2) Calculates the eligibles <br/>
3) Calculates some totals <br/>
4) Rounds the calculated amount to the nearest individual  <br/>
5) Sets the output format  </i>

In [57]:
def calculate_eligibles(fpath, writer, migration='full', acs_years=5):
    '''Reads the data and calculates the eligibles. Writes results to worksheet
    Args:
        fpath(str): filepath for the input data
        write(pandas.ExcelWriter): where to write the output
        migration(str): 'full', 'half', or 'none'
        acs_years(int): 1 or 5
    Returns:
        None
        '''
    # Determine the sheet name for the output
    out_name = 'Potential Elig 2015' + migration + str(acs_years) + 'YR'

    # Get the data set
    combined_df = create_data_frame(fpath, migration, acs_years)

    # infant_A is population from infant column
    infant_B = 1
    # infant_C is proportion below 185% <6 from column
    infant_D = 1.2866
    infant_E = 1.16
    infant_F = 0.97

    # child_A is population from 1 to 4
    child_B = 1
    # child_C is proportion below 185% <6 from column
    child_D = 1.2906
    child_E = 1.02
    child_F = 0.99

    # PRG_A is eligible infants
    PRG_B = 0.533
    PRG_Adj_compl = 0.9961  # PRG_C (but also noBF_B, and BF_B)
    PRG_D = 0.97

    # noBF_A is eligible infants
    #noBF_B is PRG_Adj_compl
    noBF_C = 0.302

    # BF_A is eligible infants
    #BF_B is PRG_Adj_compl
    BF_C = 0.308

    # Create the eligible dataframe
    df_elig = combined_df.iloc[:, [0, 1, 2, 3]]

    # Apply the adjustments
    elig_infants = combined_df['infants']*infant_B*combined_df['Prop Below 185% FPL <6']*infant_D*infant_E*infant_F*1.0
    df_elig['Infants'] = elig_infants
    df_elig['Children'] = combined_df['1-4 years']*child_B * \
        combined_df['Prop Below 185% FPL <6']*child_D*child_E*child_F
    df_elig['Pregnant'] = elig_infants*PRG_B*PRG_Adj_compl*PRG_D
    df_elig['Postpartum'] = elig_infants*PRG_Adj_compl*noBF_C
    df_elig['Breastfeeding'] = elig_infants*PRG_Adj_compl*BF_C


    # Create a total column
    df_elig['Total'] = df_elig.iloc[:, 4:].sum(
        axis=1, skipna=True, level=None, numeric_only=True)

    # Create a State Total row
    if acs_years == 5:
        row_total = np.concatenate([['48000', 'State All, Texas'], df_elig.iloc[
                                   :-1, :].sum(numeric_only=True).astype(int)]).tolist()
        df_elig = df_elig.append(
            dict(zip(df_elig.columns.tolist(), row_total)), ignore_index=True)
    # Round
    df_elig.iloc[:, 4:] = df_elig.iloc[:, 4:].astype(float).apply(np.round)

    df_elig.to_excel(writer, sheet_name=out_name, index=False)
    workbook = writer.book
    worksheet = writer.sheets[out_name]
    num_format = workbook.add_format({'num_format': '#,##0'})

    # Set the column width and format
    worksheet.set_column('C:C', 18, num_format)
    worksheet.set_column('E:J', 18, num_format)
    worksheet.set_column('A:A', 9)
    worksheet.set_column('B:B', 27)

<strong>Step 5:</strong> Output the potential eligibles to an excel workbook to get the whole set.

In [58]:
fpath = 'Data Files\\'  # Set the filepath for the folder with all the data
writer = pd.ExcelWriter('Potential Eligibles 2015 test.xlsx')
calculate_eligibles(fpath, writer)
writer.save()

c:\anaconda\lib\site-packages\ipykernel\__main__.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
